In [31]:
from datasets import load_dataset

dataset_name = "tcapelle/train_ds_triton_v2f2"

ds = load_dataset(dataset_name, split="train")

ds[0]

Generating train split: 100%|██████████| 864/864 [00:00<00:00, 38782.95 examples/s]


{'uuid': '6d32b21d-530f-4d1d-a2b2-bb08a0c056cc',
 'file_name': 'triton_matric_matmul.py',
 'repo_name': 'elphinkuo/fast_matrix_multiplication',
 'file_path': 'dot_product/on_gpu/Triton/triton_matric_matmul.py',
 'commit_hash': '4e875a17e95b7ccf9af102d2c0f8cc31ed9a29f3',
 'starcount': 0,
 'input': "@triton.jit\ndef _matmul_kernel(A, B, C, M, N, K, **meta):\n    TILE_M = meta['BLOCK_M']\n    TILE_N = meta['BLOCK_N']\n    TILE_K = 128\n    m = tl.program_id(0) * TILE_M + tl.arange(0, TILE_M)\n    n = tl.program_id(1) * TILE_N + tl.arange(0, TILE_N)\n    acc = tl.zeros((TILE_M, TILE_N), dtype=tl.float32)\n    for k in range(0, K, TILE_K):\n        a = tl.load(A + m[:, None] * K + k, mask=[m[:, None] < M, None],\n            other=0.0)\n        b = tl.load(B + k * N + n, mask=[None, n < N], other=0.0)\n        acc += tl.dot(a, b)\n    tl.store(C + m[:, None] * N + n, acc, mask=[m[:, None] < M, n < N])\n",
 'category': {'Data Type': ['fp32'],
  'Functionality': ['Matrix Multiplication', 'Ele

In [32]:
ds = ds.remove_columns("pytorch_code_with_tests")

In [33]:
ds = ds.rename_column("format_pt_code", "pt_code")

In [34]:
ds[0]

{'uuid': '6d32b21d-530f-4d1d-a2b2-bb08a0c056cc',
 'file_name': 'triton_matric_matmul.py',
 'repo_name': 'elphinkuo/fast_matrix_multiplication',
 'file_path': 'dot_product/on_gpu/Triton/triton_matric_matmul.py',
 'commit_hash': '4e875a17e95b7ccf9af102d2c0f8cc31ed9a29f3',
 'starcount': 0,
 'input': "@triton.jit\ndef _matmul_kernel(A, B, C, M, N, K, **meta):\n    TILE_M = meta['BLOCK_M']\n    TILE_N = meta['BLOCK_N']\n    TILE_K = 128\n    m = tl.program_id(0) * TILE_M + tl.arange(0, TILE_M)\n    n = tl.program_id(1) * TILE_N + tl.arange(0, TILE_N)\n    acc = tl.zeros((TILE_M, TILE_N), dtype=tl.float32)\n    for k in range(0, K, TILE_K):\n        a = tl.load(A + m[:, None] * K + k, mask=[m[:, None] < M, None],\n            other=0.0)\n        b = tl.load(B + k * N + n, mask=[None, n < N], other=0.0)\n        acc += tl.dot(a, b)\n    tl.store(C + m[:, None] * N + n, acc, mask=[m[:, None] < M, n < N])\n",
 'category': {'Data Type': ['fp32'],
  'Functionality': ['Matrix Multiplication', 'Ele

In [35]:
ds

Dataset({
    features: ['uuid', 'file_name', 'repo_name', 'file_path', 'commit_hash', 'starcount', 'input', 'category', 'licenses', 'github_url', 'description', 'pt_code', 'entrypoint', 'pt_code_runs', 'stop_reason', 'pt_code_without_tests', 'tests', 'stdout', 'stderr', 'runtime'],
    num_rows: 864
})

In [36]:
import re

def get_test_signature(test_code):
    # use a re to find the test_* items
    return re.findall(r"test_[^ ]+", test_code)

In [37]:
def check_tests_signature(row):
    "We should have a test that match the entrypoint -> test_<entrypoint>"
    tests = row["tests"]
    if tests == "":
        print("Error: tests is empty")
        return False
    entrypoint = row["entrypoint"]
    if f"test_{entrypoint}" not in tests:
        actual_tests = get_test_signature(tests)
        print(f"Error: test_{entrypoint} not in tests")
        print(f"Actual tests: {actual_tests}")
        return False
    return True



In [41]:
fds = ds.filter(lambda x: check_tests_signature(x))

Filter: 100%|██████████| 864/864 [00:00<00:00, 27910.56 examples/s]

Error: test_fwd_prepare_wy_repr not in tests
Actual tests: []


In [43]:
fds.push_to_hub(dataset_name, commit_message="fix_test_signature")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.31it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/tcapelle/train_ds_triton_v2f2/commit/f96471b74ac0da3a4d527e9b51a2f1b0121d3721', commit_message='fix_test_signature', commit_description='', oid='f96471b74ac0da3a4d527e9b51a2f1b0121d3721', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tcapelle/train_ds_triton_v2f2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tcapelle/train_ds_triton_v2f2'), pr_revision=None, pr_num=None)